# Migrationshintergrund


In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.io as pio

In [ ]:
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

## Import Data

In [ ]:
file_path = "data.xlsx"
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")
sheets.pop(next(iter(sheets)))

updated_sheets = {}

for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()

    migration_row = df[
        (df["Variable"] == "Migrationshintergrund") & 
        (df["Category"] == "Migrationshintergrund (%)")
    ]

    total_students_row = df[
        (df["Variable"] == "Migrationshintergrund") & 
        (df["Category"] == "Anzahl")
    ]

    common_columns = migration_row.columns.intersection(total_students_row.columns)[2:]
 
    migration_values = migration_row[common_columns].astype(float).fillna(0).squeeze()
    total_students_values = total_students_row[common_columns].astype(float).fillna(0).squeeze()
    
    abs_migration_students_per_studiengang = (migration_values / 100) * total_students_values

    new_row = pd.DataFrame([["Migrationshintergrund", "Absolute Anzahl"] + abs_migration_students_per_studiengang.tolist()], columns=df.columns)

    df = pd.concat([df, new_row], ignore_index=True)

    updated_sheets[jahr] = df


In [ ]:
gewichteter_anteil = {}

for jahr, df in updated_sheets.items():
    abs_migration_row = df[
        (df["Variable"] == "Migrationshintergrund") & 
        (df["Category"] == "Absolute Anzahl")
    ]

    total_students_row = df[
        (df["Variable"] == "Migrationshintergrund") & 
        (df["Category"] == "Anzahl")
    ]

    if abs_migration_row.empty or total_students_row.empty:
        print(f"Warnung: Keine Migrationshintergrund-Daten für {jahr} gefunden!")
        continue  

    common_columns = abs_migration_row.columns.intersection(total_students_row.columns)[2:]

    abs_migration_total = abs_migration_row[common_columns].sum().sum()
    total_students = total_students_row[common_columns].sum().sum()

    if total_students > 0:
        gewichteter_anteil[jahr] = (abs_migration_total / total_students) * 100
    else:
        gewichteter_anteil[jahr] = 0 

migrations_df = pd.DataFrame({
    "Jahr": list(gewichteter_anteil.keys()),
    "Gewichteter Relativer Anteil (%)": list(gewichteter_anteil.values())
})

migrations_df = migrations_df.sort_values(by="Jahr")

In [ ]:
fig = px.line(
    migrations_df, 
    x="Jahr", 
    y="Gewichteter Relativer Anteil (%)", 
    markers=True, 
    title="Der Fachbereich 09 begeistert immer mehr Studierende mit Migrationshintergrund",
    subtitle="Entwicklung der Studierendenbefragung zwischen 2013 und 2024",
    color_discrete_sequence=[px.colors.qualitative.Safe[5]],
    template="infoviz"
)

fig.update_traces(line=dict(width=3))

fig.update_layout(
    yaxis=dict(range=[0, 42], title="Studierende mit Migrationshintergrund (%)"),
)

fig.show()

## Master vs Bachelor

In [ ]:
gewichteter_anteil_bachelor = {}
gewichteter_anteil_master = {}

for jahr, df in updated_sheets.items():
    abs_migration_row = df[
        (df["Variable"] == "Migrationshintergrund") & 
        (df["Category"] == "Absolute Anzahl")
    ]

    total_students_row = df[
        (df["Variable"] == "Migrationshintergrund") & 
        (df["Category"] == "Anzahl")
    ]

    if abs_migration_row.empty or total_students_row.empty:
        print(f"Warnung: Keine Migrationshintergrund-Daten für {jahr} gefunden!")
        continue  

    bachelor_columns = [col for col in df.columns if "Bachelor" in col]
    master_columns = [col for col in df.columns if "Master" in col]

    if bachelor_columns:
        abs_migration_bachelor = abs_migration_row[bachelor_columns].sum().sum()
        total_students_bachelor = total_students_row[bachelor_columns].sum().sum()
        gewichteter_anteil_bachelor[jahr] = (abs_migration_bachelor / total_students_bachelor) * 100 if total_students_bachelor > 0 else 0
    else:
        gewichteter_anteil_bachelor[jahr] = None

    if master_columns:
        abs_migration_master = abs_migration_row[master_columns].sum().sum()
        total_students_master = total_students_row[master_columns].sum().sum()
        gewichteter_anteil_master[jahr] = (abs_migration_master / total_students_master) * 100 if total_students_master > 0 else 0
    else:
        gewichteter_anteil_master[jahr] = None

migrations_df = pd.DataFrame({
    "Jahr": list(gewichteter_anteil_bachelor.keys()),
    "Gewichteter Relativer Anteil Bachelor (%)": list(gewichteter_anteil_bachelor.values()),
    "Gewichteter Relativer Anteil Master (%)": list(gewichteter_anteil_master.values())
})
migrations_df["Jahr"] = migrations_df["Jahr"].astype(int)
migrations_df = migrations_df.sort_values(by="Jahr")

In [ ]:
fig = px.line(
    migrations_df, 
    x="Jahr", 
    y=["Gewichteter Relativer Anteil Bachelor (%)", "Gewichteter Relativer Anteil Master (%)"], 
    markers=True, 
    title="In den <span style='color:#cc5b6e;'>Masterstudiengängen</span> wächst der Anteil Studierender mit Migrationshintergrund besonders stark",
    subtitle="Entwicklung der Studierendenbefragung zwischen 2013 und 2024 im FB09",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_traces(line=dict(width=3))
last_year = migrations_df["Jahr"].max()
last_bachelor_value = migrations_df[migrations_df["Jahr"] == last_year]["Gewichteter Relativer Anteil Bachelor (%)"].values[0]
last_master_value = migrations_df[migrations_df["Jahr"] == last_year]["Gewichteter Relativer Anteil Master (%)"].values[0]

fig.add_annotation(
    x=last_year, 
    y=last_bachelor_value + 3, 
    text=f"Bachelor {last_bachelor_value:.2f}%", 
    showarrow=False, 
    ax=40, 
    ay=-10,
    font=dict(size=14, color=px.colors.qualitative.Safe[0], weight="bold")
)

fig.add_annotation(
    x=last_year, 
    y=last_master_value + 3, 
    text=f"Master {last_master_value:.2f}%", 
    showarrow=False, 
    ax=40, 
    ay=10,
    font=dict(size=14, color=px.colors.qualitative.Safe[1], weight="bold")
)

fig.update_layout(
    yaxis=dict(range=[-0.08, 55], title="Studierende mit Migrationshintergrund (%)"),
    xaxis=dict(range=[2012.9, 2024.05], title="", dtick=1),
    showlegend=False 
)

fig.show()